In [67]:
import numpy as np
import time
import random


In [184]:
class Individual:

  def __init__(self, size,chromosome):
    self.chromosome = chromosome
    self.size = size
    self.cost = np.inf

  def mutation(self, mut_prob):
    if random.random() <= mut_prob:
      mp1 = random.randint(0, self.size - 1)
      mp2 = random.randint(0, self.size - 1)
      self.chromosome[mp1], self.chromosome[mp2] = self.chromosome[mp2], self.chromosome[mp1]

  @staticmethod
  def generateRandom(n):
    return Individual(n,np.random.permutation(n))

  def cutAndCrossfill(self,parent,cro_prob):

    def insertValues(child, parent, n, crossover_point):
      while(len(child.chromosome) != n):
        if(parent.chromosome[crossover_point] not in child.chromosome):
          child.chromosome = np.append(child.chromosome, parent.chromosome[crossover_point])
        crossover_point = (crossover_point + 1) % n

    if(random.random() > cro_prob):
      return self,parent
      #return Individual(self.size,self.chromosome), Individual(parent.size,parent.chromosome)
    
    n = self.size
    crossover_point = random.randint(0, n)

    child1 = Individual(n,[])
    child2 = Individual(n,[])
    child1.chromosome = self.chromosome[:crossover_point]
    child2.chromosome = parent.chromosome[:crossover_point]

    insertValues(child1, parent, n, crossover_point)
    insertValues(child2, self, n, crossover_point)

    return (child1, child2)

  def calculate_cost(self,flow_matrix,distance_matrix):  
    cost = 0
    for i in range(self.size):
      for j in range(self.size):
        cost = cost + distance_matrix[i][j]*flow_matrix[self.chromosome[i]][self.chromosome[j]]
    self.cost = cost
    


  def __str__(self):
    return str(self.chromosome)

In [230]:
def bestTwoOutOfFive(population):
    # Select 5 individuals randomly
    tournament = [population[random.randint(
        0, len(population)-1)] for _ in range(len(population)//5)]

    # return best 2 of five
    return sorted(tournament, key=lambda x: x.cost, reverse=False)[:2]


def replaceWorst(population, newIndividual):
  population = sorted(population,key= lambda x: x.cost,reverse=True)
  if(newIndividual.cost < population[-1].cost):
    population[-1] = newIndividual


In [238]:
def geneticAlgorithm(f, d, pop_size, cro_prob, mut_prob, ter_cond, bk_fitness):
    """
    Algoritmo genetico
    """
    # Variables Initialization
    generations = 1
    n = len(f[0])
    offspring = []
    #world = []

    # INITIALISE population with random candidate solutions
    population = [Individual.generateRandom(n) for _ in range(pop_size)]

    # EVALUATE each candidate        
    for ind in population:
        ind.calculate_cost(f,d)

    # Repeat until termination condition is satisfied
    while(generations < ter_cond + 1):
        #print(f"Generacion {generations}")
        # If optimal solution exists and it is reached
        if((bk_fitness is not None) and population[0].cost == bk_fitness):
            break

        
        for i in range(pop_size//2):

            # PARENT SELECTION (best two out of five)
            father, mother = bestTwoOutOfFive(population)

            # CROSSOVER
            child1, child2 = father.cutAndCrossfill(mother, cro_prob)

            # MUTATION
            child1.mutation(mut_prob)
            child2.mutation(mut_prob)

            # EVALUATION OF NEW CANDIDATES
            child1.calculate_cost(f,d)
            child2.calculate_cost(f,d)


            offspring.append(child1)
            offspring.append(child2)


        #world = population + offspring

        # SURVIVOR SELECTION
        population = sorted(offspring, key= lambda x: x.cost, reverse=False)[:pop_size]
        offspring = []
        #world = []

        
        # replaceWorst(population, child1)
        # replaceWorst(population, child2)

        generations += 1

    return (generations-1, population)


In [239]:
def read_qap_dataset(filename):
    def read_integers(filename):
        with open(filename) as f:
            return [int(elem) for elem in f.read().split()]

    file_it = iter(read_integers(filename))
    # Number of points
    n = next(file_it)
    # Distance between locations
    A = [[next(file_it) for j in range(n)] for i in range(n)]
    # Flow between factories
    B = [[next(file_it) for j in range(n)] for i in range(n)]

    return (n, B, A)


In [240]:
def test(pop_size, cro_prob, mut_prob, ter_cod, runs, filename, best_cost=None):
    runs_stats = []

    n, f, d = read_qap_dataset(filename)

    for i in range(runs):
        print('Run number: {}'.format(i))
        begining = time.time()
        g,p = geneticAlgorithm(f,d,pop_size,cro_prob,mut_prob,ter_cod,best_cost)
        end = time.time()

        total_time = end - begining
        print(f'Total time: {total_time}')
        best = p[0]
        worst = p[-1]
        average_cost = np.mean([ind.cost for ind in p])
        runs_stats.append({ 'runtime': total_time, 'best': best, 'worst': worst, 'average_cost': average_cost })
    
    costs = [ stat['average_cost'] for stat in runs_stats ]
    times = [ stat['runtime'] for stat in runs_stats ]
    #bests_costs = [ stat['best'].cost for stat in runs_stats ]
    #worsts_costs = [ stat['worst'].cost for stat in runs_stats ]
    best_individual = min([ stat['best'] for stat in runs_stats ], key=lambda x: x.cost)
    worst_individual = max([ stat['worst'] for stat in runs_stats ], key=lambda x: x.cost)
    cost_average = np.mean(costs)
    cost_std = np.std(costs)
    time_average = np.mean(times)
    time_std = np.std(times)
    #max_cost = max(costs)
    #min_cost = min(costs)
    max_time = max(times)
    min_time = min(times)
    #best_of_bests = min(bests_costs)
    #worst_of_worst = max(worsts_costs)

    print(f'Costo promedio: {cost_average}')
    print(f'Desviación estandar de costos: {cost_std}')
    print(f'Mejor solución encontrada: {best_individual}: {best_individual.cost}')
    print(f'Peor solución encontrada: {worst_individual}: {worst_individual.cost}')
    print(f'Tiempo promedio de ejecución: {time_average}')
    print(f'Desviación estandar tiempo de ejecución: {time_std}')
    print(f'Valor máximo (tiempo de ejecución): {max_time}')
    print(f'Valor mínimo (tiempo de ejecución): {min_time}')


## Parámetros 1
* Tamaño de población: 300
* Número máximo de generaciones: 50,000
* Probabilidad de cruzamiento: 0.9
* Probabilidad de mutación: 0.2
* Número de corridas: 30
* Dataset: lipa20a

In [243]:
POP_SIZE = 300
CRO_PROB = 1
MUT_PROB = 0.8
TER_COND = 1000
RUNS = 1    
FILENAME = 'chr12a.dat'

#test(POP_SIZE,CRO_PROB,MUT_PROB,TER_COND,RUNS,FILENAME,best_cost=1160)
n, f, d = read_qap_dataset(FILENAME)
g,p = geneticAlgorithm(f,d,POP_SIZE,CRO_PROB,MUT_PROB,TER_COND,bk_fitness=None)

for e in p:
    print(e,e.cost)


#test(POP_SIZE,CRO_PROB,MUT_PROB,TER_COND,RUNS,FILENAME,best_cost=3683)

[ 9  7  6  1  0 10  8  4  5  2 11  3] 15932
[ 9  7  6  1  0 10  8  4  5  2 11  3] 15932
[ 9  7  6  1  0 10  8  4  5  2 11  3] 15932
[ 9  7  6  1  0 10  8  4  5  2 11  3] 15932
[ 9  7  6  1  0 10  8  4  5  2 11  3] 15932
[ 9  7  6  1  0 10  8  4  5  2 11  3] 15932
[ 9  7  6  1  0 10  8  4  5  2 11  3] 15932
[ 9  7  6  1  0 10  8  4  5  2 11  3] 15932
[ 9  7  6  1  0 10  8  4  5  2 11  3] 15932
[ 9  7  6  1  0 10  8  4  5  2 11  3] 15932
[ 9  7  6  1  0 10  8  4  5  2 11  3] 15932
[ 9  7  6  1  0 10  8  4  5  2 11  3] 15932
[ 9  7  6  1  0 10  8  4  5  2 11  3] 15932
[ 9  7  6  1  0 10  8  4  5  2 11  3] 15932
[ 9  7  6  1  0 10  8  4  5  2 11  3] 15932
[ 9  7  6  1  0 10  8  4  5  2 11  3] 15932
[ 9  7  6  1  0 10  8  4  5  2 11  3] 15932
[ 9  7  6  1  0 10  8  4  5  2 11  3] 15932
[ 9  7  6  1  0 10  8  4  5  2 11  3] 15932
[ 9  7  6  1  0 10  8  4  5  2 11  3] 15932
[ 9  7  6  1  0 10  8  4  5  2 11  3] 15932
[ 9  7  6  1  0 10  8  4  5  2 11  3] 15932
[ 9  7  6  1  0 10  8  4  5  2 1

In [204]:
asd = [40, 2, 19, 23, 24, 7, 34,  3, 39, 14, 20, 15,  1, 10, 11, 17, 18, 28, 33, 16,30, 6, 12, 36,  9, 4, 22, 32, 21,  8, 29, 25, 38, 37,  5, 13, 26, 27, 35, 31]
FILENAME = 'tho40.dat'
indiv = Individual(40,[i-1 for i in asd])
n, f, d = read_qap_dataset(FILENAME)

indiv.calculate_cost(f,d)

print(indiv.cost)

# p1 = Individual.generateRandom(9)
# p2 = Individual.generateRandom(9)




240516


**Recursos**

**Test cases**
https://neos-guide.org/content/quadratic-assignment-problem#:~:text=The%20quadratic%20assignment%20problem%20(QAP)%20was%20introduced%20by%20Koopmans%20and,minimize%20the%20total%20assignment%20cost

**Datasets**
http://www.mgi.polymtl.ca/anjos/qaplib/inst.html

**Todo**

**Prioridad Alta**
*   Esqueleto Clase (Dani) ✅✅
*   Esqueleto Algoritmo (Morfin) ✅✅
*   Mutación (Dani) ✅✅
*   Cruzamiento (Morfin) ✅✅
*   Selección de Padres (Dani) ✅✅
*   Reemplazo (Morfin) ✅✅
*   Fitness (Dani) ✅

**Prioridad Media**
*   Leer el archivo (Ambos) ✅✅
*   Calcular STD, AVG, max, min (Dani)✅
*   Guardar en archivo la mejor solución (Dani)

**Prioridad Baja**
*   Calibrar parámetros

**Necesario**
*   Redactar el reporte

In [4]:
n

20